In [1]:
import csv
import numpy as np
import pandas as pd
import time
from urllib2 import urlopen # python2
import urllib2
#import urllib.request #python3
#from urllib.request import urlopen
#import urllib.parse


In [3]:
raw_sider = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/meddra_all_label_se_header.tsv")

In [2]:
stitch_cid = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/STITCH_CID.tsv")

In [4]:
print(len(raw_sider))

4753141


In [5]:
raw_sider_pt = raw_sider[raw_sider.MedDRA_concept_type == 'PT']
print(len(raw_sider_pt))
raw_sider_pt.head(1)

2523626


,source_label,STITCH_compound_id_flat,STITCH_compound_id_stereo,UMLS_concept_id,MedDRA_concept_type,UMLS_concept_id_for_MedDRA,side_effect_name
1,EMA/WC500020092.html,CID100216416,CID000216416,C0000737,PT,C0000737,Abdominal pain


In [6]:
print(len(raw_sider_pt.drop_duplicates(subset = ['STITCH_compound_id_flat','STITCH_compound_id_stereo', 'UMLS_concept_id_for_MedDRA', 'side_effect_name'])))

152759


In [1]:
sider = raw_sider_pt[['STITCH_compound_id_flat','STITCH_compound_id_stereo', 'UMLS_concept_id_for_MedDRA', 'side_effect_name']]

NameError: name 'raw_sider_pt' is not defined

In [8]:
print(len(sider))

2523626


In [9]:
sider_wo_dup = sider.drop_duplicates()
print(len(sider_wo_dup))
sider_wo_dup.head()

152759


,STITCH_compound_id_flat,STITCH_compound_id_stereo,UMLS_concept_id_for_MedDRA,side_effect_name
1,CID100216416,CID000216416,C0000737,Abdominal pain
2,CID100216416,CID000216416,C0687713,Gastrointestinal pain
4,CID100216416,CID000216416,C0002170,Alopecia
6,CID100216416,CID000216416,C0002395,Dementia Alzheimer's type
8,CID100216416,CID000216416,C0002622,Amnesia


In [42]:
len(sider_wo_dup.drop_duplicates(subset='side_effect_name'))

4251

In [10]:
stitch_cid.head(1)

,flat_chemical,stereo_chemical,source_name,source_id
0,CID100000001,CID000000001,PC,1


In [11]:
sider_wo_dup2 = sider_wo_dup.drop_duplicates(subset=['STITCH_compound_id_flat', 'STITCH_compound_id_stereo'])

In [12]:
print(len(sider_wo_dup2))
sider_wo_dup2.head()

1556


,STITCH_compound_id_flat,STITCH_compound_id_stereo,UMLS_concept_id_for_MedDRA,side_effect_name
1,CID100216416,CID000216416,C0000737,Abdominal pain
653,CID100060795,CID000060795,C0002170,Alopecia
894,CID100001690,CID000031703,C0000737,Abdominal pain
1276,CID105251896,CID006918537,C0002994,Angioedema
1346,CID100004666,CID000441276,C0000731,Abdominal distension


In [37]:
stitch_cid[(stitch_cid.flat_chemical == 'CID100216416') & (stitch_cid.stereo_chemical == 'CID000216416')]

,flat_chemical,stereo_chemical,source_name,source_id
726186,CID100216416,CID000216416,PC,10671241
8282091,CID100216416,CID000216416,PC,216416
9947949,CID100216416,CID000216416,PC,23724939
13672549,CID100216416,CID000216416,PC,3081919
41347226,CID100216416,CID000216416,PC,68514402
41350863,CID100216416,CID000216416,PC,68517869
41633260,CID100216416,CID000216416,PC,68779769


In [38]:
sider_wo_dup2[(sider_wo_dup2.STITCH_compound_id_flat=='CID100216416') & (sider_wo_dup2.STITCH_compound_id_stereo == 'CID000216416')]

,STITCH_compound_id_flat,STITCH_compound_id_stereo,UMLS_concept_id_for_MedDRA,side_effect_name
1,CID100216416,CID000216416,C0000737,Abdominal pain


In [14]:
query_pair = list()
for index, row in sider_wo_dup2.iterrows():
    record =row.tolist()
    
    #found = stitch_cid.query('@record[0] == flat_chemical and @record[1]==stereo_chemical')
    found = stitch_cid[(stitch_cid.flat_chemical == record[0]) & (stitch_cid.stereo_chemical == record[1])]
    query_pc_id = found['source_id'].tolist()
    
    query_pair.append([record[0], record[1], query_pc_id])

In [23]:
df_query_pair = pd.DataFrame()
for i in query_pair:
    for j in i[2]:
        temp_df = pd.DataFrame([[i[0], i[1], j]], columns = ['stitch_flat', 'stitch_stereo', 'pubchem_cid'])
        df_query_pair = df_query_pair.append(temp_df)
df_query_pair.to_csv("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/1_sider_pubchem.tsv", sep='\t', index=False)  
print(len(df_query_pair))

In [13]:
df_query_pair = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/1_sider_pubchem.tsv")

In [14]:
print(len(df_query_pair))
df_query_pair.head()

13265


,stitch_flat,stitch_stereo,pubchem_cid
0,CID100216416,CID000216416,10671241
1,CID100216416,CID000216416,216416
2,CID100216416,CID000216416,23724939
3,CID100216416,CID000216416,3081919
4,CID100216416,CID000216416,68514402


In [18]:
# using UniChem REST API, convert pubchemID to DrugBank ID
with open("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/3_SIDER_DrugBank_id.tsv", "wa") as resultFile:
    features = ['STITCH_compound_id_flat', 'STITCH_compound_id_stereo', 'PubChem_ID', 'DrugBank_ID']
    resultTsv = csv.DictWriter(resultFile, features, dialect=csv.excel_tab)
    resultTsv.writeheader()
    
    #cnt = 0
    url_base = 'https://www.ebi.ac.uk/unichem/rest/src_compound_id/'
    print'%d : ' % cnt + time.strftime('%a %H:%M:%S')
    for index, row in df_query_pair[cnt-1:].iterrows():
        record = row.tolist()
        stitch_id_flat = record[0]
        stitch_id_stereo = record[1]
        pubchem_id = record[2]
        url = url_base + str(pubchem_id) + '/22/2'

        try:
            f = urlopen(url)
        except urllib2.HTTPError, e:
            if e.code == 404:
                cnt = cnt + 1
                if cnt % 1000 == 0 :
                    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
                continue
            else:
                print 'e.code : %d, url : %s, cnt : d' % (e.code, url, cnt)
                raise
        except urllib2.URLError, e:
            if e.args[0][0] == 110: # connection time out
                #sleep_secs = attempt ** 2
                print e
                print url
                print '  cnt = %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                print '    Retrying in %d seconds...' % sleep_secs
                time.sleep(sleep_secs)
                cnt= cnt +1
                if cnt % 1000 == 0 :
                    print '%d : '% cnt + time.strftime('%a %H:%M:%S')
                continue 
            else:
                print e
                print '   cnt : %d, error time : ' % (cnt) + time.strftime('%a %H:%M:%S')
                raise
        else:          
            all_data = f.readlines()
            if all_data[0] == '[]':
                cnt = cnt + 1
            else:
                #db_id=all_data[0].decode("utf-8").split(':')[1]  
                db_id=all_data[0] 
                resultTsv.writerow({'STITCH_compound_id_flat':stitch_id_flat, 'STITCH_compound_id_stereo':stitch_id_stereo, 
                                'PubChem_ID' : pubchem_id, 'DrugBank_ID' : db_id})     
                cnt = cnt + 1
            if cnt % 1000 == 0 :
                print '%d : '% cnt + time.strftime('%a %H:%M:%S')

print time.strftime('%a %H:%M:%S')


3301 : Mon 01:08:23
4000 : Mon 01:20:41
5000 : Mon 01:38:19
6000 : Mon 01:55:51
7000 : Mon 02:13:22
8000 : Mon 02:30:53
9000 : Mon 02:48:26
10000 : Mon 03:05:58
11000 : Mon 03:23:29
12000 : Mon 03:40:59
13000 : Mon 03:58:33
Mon 04:03:13


In [23]:
df_query_pair.head(1)

,stitch_flat,stitch_stereo,pubchem_cid
0,CID100216416,CID000216416,10671241


In [24]:
print(len(df_query_pair))
print(len(df_query_pair.drop_duplicates(subset=['stitch_flat', 'stitch_stereo'])))

13265
1556


In [20]:
df_raw = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/3_SIDER_DrugBank_id.tsv")

In [43]:
print(len(df_raw))
print(len(df_raw.drop_duplicates()))
print(len(df_raw.drop_duplicates(subset=['STITCH_compound_id_flat', 'STITCH_compound_id_stereo'])))
df_raw.head()

861
861
787


,STITCH_compound_id_flat,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
0,CID100002656,CID005479530,5479530,"[{""src_compound_id"":""DB01212""}]"
1,CID100002650,CID005481173,5481173,"[{""src_compound_id"":""DB00438""}]"
2,CID100002650,CID005481173,57514439,"[{""src_compound_id"":""DB00438""}]"
3,CID100002771,CID000002771,2771,"[{""src_compound_id"":""DB00215""}]"
4,CID100002617,CID000033255,33255,"[{""src_compound_id"":""DB01327""}]"


In [29]:
result = pd.DataFrame()
for index, row in df_raw.iterrows():
    record = row.tolist()
    db_ids = record[3].split(',')
    for i in db_ids:
        db_id = i.split(':')[1].lstrip('\"').rstrip('\"}]')
        temp_df = pd.DataFrame([[record[0], record[1], record[2], db_id]], columns = ['STITCH_compound_id_flat',
                                                                                      'STITCH_compound_id_stereo', 
                                                                                      'PubChem_ID', 'DrugBank_ID'])
        result = result.append(temp_df)
result.index = range(0, len(result))
    
        

In [32]:
print len(result)
print len(result.drop_duplicates())
print len(result.drop_duplicates(subset = ['STITCH_compound_id_flat','STITCH_compound_id_stereo', 'DrugBank_ID']))
print len(result.drop_duplicates(subset = ['STITCH_compound_id_flat','STITCH_compound_id_stereo']))
print len(result.drop_duplicates(subset = ['DrugBank_ID']))

874
874
801
787
801


In [33]:
result.to_csv("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/3_2_SIDER_DrugBank_id.tsv", sep='\t', index=False)

In [34]:
result.head()

,STITCH_compound_id_flat,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
0,CID100002656,CID005479530,5479530,DB01212
1,CID100002650,CID005481173,5481173,DB00438
2,CID100002650,CID005481173,57514439,DB00438
3,CID100002771,CID000002771,2771,DB00215
4,CID100002617,CID000033255,33255,DB01327


In [45]:
result[result.duplicated(subset = ['STITCH_compound_id_flat','STITCH_compound_id_stereo', 'DrugBank_ID'], keep=False)].sort_values(by = 'PubChem_ID')#.sort_values(by = 'DrugBank_ID')

,STITCH_compound_id_flat,STITCH_compound_id_stereo,PubChem_ID,DrugBank_ID
337,CID100000119,CID000000119,119,DB02530
482,CID100000564,CID000000564,564,DB00513
670,CID100000750,CID000000750,750,DB00145
177,CID100002141,CID000002141,2141,DB01143
443,CID100002266,CID000002266,2266,DB00548
5,CID100002764,CID000002764,2764,DB00537
6,CID100002764,CID000002764,2764,DB05488
209,CID100002767,CID000002767,2767,DB00515
68,CID100003446,CID000003446,3446,DB00996
66,CID100004539,CID000004539,4539,DB01059


In [36]:
stitch_db_table = result[['STITCH_compound_id_flat','STITCH_compound_id_stereo', 'DrugBank_ID']].drop_duplicates()

In [38]:
print len(stitch_db_table)
print len(result[['STITCH_compound_id_flat','STITCH_compound_id_stereo', 'DrugBank_ID']])

801
874


In [39]:
stitch_db_table.to_csv("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/3_3_SIDER_STITCHid_DrugBankid.tsv", sep='\t', index=False)

In [47]:
stitch_db_table[stitch_db_table.duplicated(subset=['STITCH_compound_id_flat','STITCH_compound_id_stereo'], keep=False)]

,STITCH_compound_id_flat,STITCH_compound_id_stereo,DrugBank_ID
5,CID100002764,CID000002764,DB00537
6,CID100002764,CID000002764,DB05488
25,CID100005203,CID000063009,DB01104
26,CID100005203,CID000063009,DB08567
62,CID100004454,CID000008378,DB00452
63,CID100004454,CID000008378,DB00994
128,CID100004428,CID005360515,DB05302
129,CID100004428,CID005360515,DB05508
130,CID100004428,CID005360515,DB00704
167,CID100002524,CID005280453,DB00136
